### 어휘 집합 구축하기

In [ ]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 2.8 MB 8.5 MB/s 
     |████████████████████████████████| 582 kB 61.6 MB/s 
     |████████████████████████████████| 57 kB 7.0 MB/s 
     |████████████████████████████████| 140 kB 74.0 MB/s 
     |████████████████████████████████| 419 kB 72.8 MB/s 
     |████████████████████████████████| 596 kB 56.0 MB/s 
     |████████████████████████████████| 880 kB 63.5 MB/s 
     |████████████████████████████████| 3.3 MB 44.5 MB/s 
     |████████████████████████████████| 101 kB 14.1 MB/s 
     |████████████████████████████████| 1.1 MB 56.6 MB/s 
     |████████████████████████████████| 96 kB 6.5 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 271 kB 75.3 MB/s 
     |████████████████████████████████| 144 kB 78.0 MB/s 
  Created wheel for sacremoses: filename

In [ ]:
#구글 드라이브 연동
from google.colab import drive
drive.mount('/gdrive', force_remount=True) #구축할 어휘집합을 따로 저장해두기위해 구글 드라이브 연동시킴

Mounted at /gdrive


### BPE 기반 토크나이저 만들기
- 말뭉치 내려받기 및 전처리

In [ ]:
#nsmc 영화 리뷰 다운로드
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 87.9MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 23.3MB/s]                            


In [ ]:
#nsmc에 포함된 영화 리뷰들을 순수 텍스트 형태로 코랩 환경 로컬의 지정된 디렉터리에 저장해둠
import os
def write_lines(path, lines):
  with open(path, 'w', encoding='utf-8') as f:
    for line in lines:
      f.write(f'{line}\n')

write_lines("/content/train.txt", nsmc.train.get_all_texts())
write_lines("/content/test.txt", nsmc.test.get_all_texts())

### GPT 토크나이저 구축

- GPT 토크나이저 기법 BPE - 유니코드 바이트 기준 BPE 사용 
-> 어휘 집합 구축 대상 말뭉치를 유니코드..로 변환하고 이들을 문자 취급해 가장 자주 등장한 문자열을 병합하는 방식으로 어휘 집합을 만든다는 의미 

In [ ]:
f = open("/content/train.txt",'r')
lines = f.readlines()
for line in lines[:20]:
    print(line)

아 더빙.. 진짜 짜증나네요 목소리

흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나

너무재밓었다그래서보는것을추천한다

교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정

사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다

막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.

원작의 긴장감을 제대로 살려내지못했다.

별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네

액션이 없는데도 재미 있는 몇안되는 영화

왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?

걍인피니트가짱이다.진짜짱이다♥

볼때마다 눈물나서 죽겠다90년대의 향수자극!!허진호는 감성절제멜로의 달인이다~

울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해

담백하고 깔끔해서 좋다. 신문기사로만 보다 보면 자꾸 잊어버린다. 그들도 사람이었다는 것을.

취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지

ㄱ냥 매번 긴장되고 재밋음ㅠㅠ

참 사람들 웃긴게 바스코가 이기면 락스코라고 까고바비가 이기면 아이돌이라고 깐다.그냥 까고싶어서 안달난것처럼 보인다

굿바이 레닌 표절인것은 이해하는데 왜 뒤로 갈수록 재미없어지냐

이건 정말 깨알 캐스팅과 질퍽하지않은 산뜻한 내용구성이 잘 버무러진 깨알일드!!♥

약탈자를 위한 변명, 이라. 저놈들은 착한놈들 절대 아닌걸요.



In [ ]:
import os
os.getcwd()
os.makedirs("/gdrive/My Drive/nlpbook/bbpe", exist_ok = True)

'/content'

In [ ]:
#바이트 수준 BPE 어휘 집합 구축
from tokenizers import ByteLevelBPETokenizer

bytebpe_tokenizer = ByteLevelBPETokenizer() 
bytebpe_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"],  #학습 말뭉치를 리스트 형태로 넣기
    vocab_size = 10000, #어휘 집합 크기 조절
    special_tokens = ["[PAD]"] #특수 토큰 추가!
)
bytebpe_tokenizer.save_model("/gdrive/My Drive/nlpbook/bbpe")

['/gdrive/My Drive/nlpbook/bbpe/vocab.json',
 '/gdrive/My Drive/nlpbook/bbpe/merges.txt']

vocab.json : 바이트 수준 BPE의 어휘 집합  
merges.txt : 바이그램 쌍의 병합 우선순위
   

In [ ]:
import json 

with open('/gdrive/My Drive/nlpbook/bbpe/vocab.json') as json_file:
    json_data = json.load(json_file)

In [ ]:
json_data #1000개의 어휘집합

{'[PAD]': 0,
 '!': 1,
 '"': 2,
 '#': 3,
 '$': 4,
 '%': 5,
 '&': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '*': 10,
 '+': 11,
 ',': 12,
 '-': 13,
 '.': 14,
 '/': 15,
 '0': 16,
 '1': 17,
 '2': 18,
 '3': 19,
 '4': 20,
 '5': 21,
 '6': 22,
 '7': 23,
 '8': 24,
 '9': 25,
 ':': 26,
 ';': 27,
 '<': 28,
 '=': 29,
 '>': 30,
 '?': 31,
 '@': 32,
 'A': 33,
 'B': 34,
 'C': 35,
 'D': 36,
 'E': 37,
 'F': 38,
 'G': 39,
 'H': 40,
 'I': 41,
 'J': 42,
 'K': 43,
 'L': 44,
 'M': 45,
 'N': 46,
 'O': 47,
 'P': 48,
 'Q': 49,
 'R': 50,
 'S': 51,
 'T': 52,
 'U': 53,
 'V': 54,
 'W': 55,
 'X': 56,
 'Y': 57,
 'Z': 58,
 '[': 59,
 '\\': 60,
 ']': 61,
 '^': 62,
 '_': 63,
 '`': 64,
 'a': 65,
 'b': 66,
 'c': 67,
 'd': 68,
 'e': 69,
 'f': 70,
 'g': 71,
 'h': 72,
 'i': 73,
 'j': 74,
 'k': 75,
 'l': 76,
 'm': 77,
 'n': 78,
 'o': 79,
 'p': 80,
 'q': 81,
 'r': 82,
 's': 83,
 't': 84,
 'u': 85,
 'v': 86,
 'w': 87,
 'x': 88,
 'y': 89,
 'z': 90,
 '{': 91,
 '|': 92,
 '}': 93,
 '~': 94,
 '¡': 95,
 '¢': 96,
 '£': 97,
 '¤': 98,
 '¥': 99,
 '¦':

In [ ]:
merge = open('/gdrive/My Drive/nlpbook/bbpe/merges.txt','r') 
lines = merge.readlines() #바이트 수준 BPE 병합 우선순위
for line in lines:
    print(line)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
ĠìķĦ íĮĮ

ìĭľ ê¸°

ë³´ ìĭľê¸¸

ëĮĢ ìĹĲ

ìŀĪ ìľ¼ë©´

ìĥģ ìĹĲ

Ġìĸ´ íľ´

ìĭł ëıĦ

Ġê± ±ìłķ

ëķĮ ë§¤

ìĨĮ ê°Ģ

ëĵ¤ìĿ´ ëŀĳ

ìłķë§Ĳ ë¡ľ

Ġë¯¸ ì³¤

ëĲĺ ëĤĺ

ìĦł ìĿ´

ìĦł íĥĿ

íĭ ĭ

ëıĻ ë¬¼

ĠëŃĲ ê³ł

ĠëŃĲ íķĺëĬĶ

ĠìĹŃ ëŁī

Ġë°Ķ ëĭ¤

ĠìĹĨëĬĶ ê²Į

ì²Ń ì¶ĺ

Ġìĭł ìĿĺ

Ġì£½ ìĹ¬

ĠìĿ´íķ´ íķłìĪĺ

Ġë°° íĬ¸ë§¨

ìŀĺ ë§Įëĵ¤

........ ...

íĮĮ ì¹ĺëħ¸

ĠìłĲ ìĿ´

ë¹ł ìł¸

íıī ìĥĿ

ì§ĢìķĬ ëĬĶëĭ¤

ĠìĥĿ ìĹĲ

Ġë´Ĳìķ¼ íķĺëĬĶ

ëĪĦ êµ°

ìĽĶ íķľ

ĠìĭľëĮĢ ìĹĲ

ĠìĿ´ë¦Ħ ìĿ´

Ġê°ĸ ëĭ¤

ìķĮë°Ķ ëĵ¤

Ġìŀĺëª» ëĲľ

Ġì« ĵ

Ġì¹ľêµ¬ ê°Ģ

Ġë§ĮìłĲ ìĹĲ

âĺħ âĺħ

ë¦¬ë©´ ìĦľ

ĠíĿ¡ ìŀħ

Ġê°Ģë²¼ ìļ´

ìĹĲëĮĢ íķ´

? ..

ĳ ¼

íķĺ ìĭ¤

¬ë ½ķ

Ġí ĺģ

ìĭ ¼

ëĤĺ ìģĺ

ĠëĤ ¯

ëĵ¤ íķľíħĮ

ìĿ¸ ê²Į

Ġë³´ ëĿ¼ê³ł

Ġê·¸ ê²ĥìĿ´

ĠëĤĺ ìĺ¬ëķĮ

Ġíķĺ ìĹ¬

ìķ¼ íķľëĭ¤

ì£¼ ì§Ģ

ì£¼ ìĿ¼

ìłģ ìĿ´ëĿ¼

ĠìĻ ¤ì¼Ģ

ê³¼ ê±°

ìķĺ ëĦ¤

íĶ ĶìĿ´

ì¹ĺ ìĿĺ

ĠìĨ Ķ

Ġìŀ¬ë°Į ìĹĪëįĺ

ĠìķĪ ê°Ģ

ĠëŃ £

ĠìĻľ ê³¡

íĬ¸ ë¥¼

ìłĢ ì§Ī

ëįĶ ë§¨

ë²Ħ ë¦´

Ġìľł íĸī

Ġìŀ¥ ìķł

ĠìĤ¬ëŀĮ ìĿ´ëĿ¼ë©´

ĠëĤ´ìļ© ìĿĺ

ëĭĺ ìĿĢ

Ġë§¤ ë¯¸

Ġë§¤ ëģĦ

ë§Īë ¥´

Ġ

In [ ]:
len(lines)

9744

### BERT 토크나이저 구축
- wordpiece 토크나이저 사용
- 구축순서: 학습 말뭉치 준비 -> 워드피스 어휘집합구축 결과 저장해둘 디렉토리 만들기

In [ ]:
#디렉토리 만들기
import os
os.makedirs("/gdrive/My Drive/nlpbook/wordpiece",exist_ok=True)

In [ ]:
#워드피스 어휘집합 구축
from tokenizers import BertWordPieceTokenizer 
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False) 
wordpiece_tokenizer.train(
    files = ['/content/train.txt', '/content/test.txt'],
    vocab_size = 10000,
)

wordpiece_tokenizer.save_model('/gdrive/My Drive/nlpbook/wordpiece')

['/gdrive/My Drive/nlpbook/wordpiece/vocab.txt']

In [ ]:
vocab = open('/gdrive/My Drive/nlpbook/wordpiece/vocab.txt','r') 
lines = vocab.readlines() #바이트 수준 BPE 병합 우선순위
for line in lines:
    print(line)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
굳굳

긴박

낫겠다

났다

대결

말인가

버린

본격

브라

성인이

손가락

신인

싸우

올드

원숭

이종

인류

일주일

잃은

전화

좋지

직업

페이크

허나

##진창

##어색

##어린

##간히

##라지만

##트로

##울뿐

##전의

##집어

##듯하다

##뻔했다

##느냐

##심에

##ed

##햇는데

연기하는

드라마다

보는것

다시금

마지막은

이야기의

재밌게봤어요

이영화의

##정도의

아까워서

아무나

이상해

이상은

모든것이

필요가

##00원

살아있는

보게되

행복하게

스타워즈

이미지

독특하고

1등

bad

movie

같았다

건질

게스트

끝을

능가

다이하

동영상

드네요

마다

마찬

마법

머릿

먹칠

못본

못만들

밝혀

베이

부활

불법

빠지는

싶습니다

에드

역겹다

차승

타란티노

토니

##로그

##때부터

##스톤

##인에

##있냐

##할수가

##비해

##놨네

##놈이

##ig

##ck

영화일

드라마에

드라마도

드라마중

드라마로

보는거

이거보단

##하나하나

대박이다

가족애

긴장감을

미국식

감동적이에요

관객이

귀여워요

있었고

오래전에

돌아보게

캐스팅이

##것같네요

삼류영화

아버지의

누군지

잘생겼

스파이더맨

바탕으로

is

간지

것인가

극중

담고

되돌아

마냥

멋있고

명대사

모니

뭐여

민주

새삼

섬뜩

성형

쉬운

안좋

억울

열라

오는

왕조

우린

태국

한동안

한석규

##안돼

##리스트

##어어

##어스

##수선

##것같은데

##갔는데

##님은

##후에

##바닥

##소름

##잡는

##웃겨

##선택

##존나

##못하고

##력에

##태현

##졌습니다

영화지

지루하기

사랑스럽고

아이들에게

여자를

주인공들이

##연기를

공포가

보여줄

유치해


---
### 토큰화 하기

### GPT 입력값 만들기
- GPT 입력값 만들려면 토크나이저 준비해야함 -> BPE tokenizer 사용 
- merge.txt(우선순위), vocab.json(어휘집합)   
 `bytebpe_tokenizer.train(...)` 로 파일 생성

In [ ]:
#GPT 토크나이저 선언
from transformers import GPT2Tokenizer 
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/gdrive/My Drive/nlpbook/bbpe")
tokenizer_gpt.pad_token = "[PAD]"  #더미토큰- 길이 맞춰줌!

file /gdrive/My Drive/nlpbook/bbpe/config.json not found


In [ ]:
f = open("/content/train.txt",'r')
lines = f.readlines()
line_list = []
for line in lines[:3]:
    line_list.append(line)
line_list

['아 더빙.. 진짜 짜증나네요 목소리\n',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나\n',
 '너무재밓었다그래서보는것을추천한다\n']

In [ ]:
#토큰화하기
sentences = line_list
for sentence in sentences:
  token_sen = tokenizer_gpt.tokenize(sentence)
  print(token_sen)
  print(len(token_sen))

['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬', 'Ċ']
8
['íĿł', '...', 'íı¬ìĬ¤íĦ°', 'ë³´ê³ł', 'Ġì´ĪëĶ©', 'ìĺģíĻĶ', 'ì¤Ħ', '....', 'ìĺ¤ë²Ħ', 'ìĹ°ê¸°', 'ì¡°ì°¨', 'Ġê°Ģë³į', 'ì§Ģ', 'ĠìķĬ', 'êµ¬ëĤĺ', 'Ċ']
16
['ëĦĪë¬´', 'ìŀ¬ë', '°', 'ĵ', 'ìĹĪëĭ¤', 'ê·¸ëŀĺìĦľ', 'ë³´ëĬĶ', 'ê²ĥìĿĦ', 'ì¶Ķì²ľ', 'íķľëĭ¤', 'Ċ']
11


In [ ]:
#GPT 토크나이저로 토큰화 하기
sentences = line_list
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]
tokenized_sentences

[['ìķĦ',
  'ĠëįĶë¹Ļ',
  '..',
  'Ġì§Ħì§ľ',
  'Ġì§ľì¦ĿëĤĺ',
  'ëĦ¤ìļĶ',
  'Ġëª©ìĨĮë¦¬',
  'Ċ'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ',
  'Ċ'],
 ['ëĦĪë¬´',
  'ìŀ¬ë',
  '°',
  'ĵ',
  'ìĹĪëĭ¤',
  'ê·¸ëŀĺìĦľ',
  'ë³´ëĬĶ',
  'ê²ĥìĿĦ',
  'ì¶Ķì²ľ',
  'íķľëĭ¤',
  'Ċ']]

In [ ]:
#GPT 모델 입력 만들기
batch_inputs = tokenizer_gpt(
    sentences,
    padding = 'max_length', #문장 최대 길이에 맞춰 패딩시켜줌
    max_length = 12,        #문장의 토큰 기준 최대 길이
    truncation = True,      #문장 잘림 허용 옵션
)

#### 위의 코드를 실행하면 2가지 입력값이 만들어짐
- input_ids : 토큰화 결과를 가지고 각 토큰을 인덱스로 바꾼 것!  
  vocab.json 파일 확인하면 각 어휘순서대로 나열되어 있는데 이 순서가 바로 인덱싱.
   - 인덱싱: 토큰을 인덱스로 변환하는 과정
- attention_mask : 일반 토큰이 자리한 곳(1), 패딩 토큰이 자리한 곳(0)을 구분해주는 장치

In [ ]:
batch_inputs

{'input_ids': [[334, 2338, 263, 581, 4055, 464, 3808, 199, 0, 0, 0, 0], [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292], [694, 477, 109, 242, 643, 3965, 1067, 2285, 2716, 969, 199, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]}

In [ ]:
display(batch_inputs["input_ids"])
display(batch_inputs["attention_mask"])

[[334, 2338, 263, 581, 4055, 464, 3808, 199, 0, 0, 0, 0],
 [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292],
 [694, 477, 109, 242, 643, 3965, 1067, 2285, 2716, 969, 199, 0]]

[[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]

In [ ]:
len(batch_inputs["input_ids"])

3

### BERT 입력값 만들기
- BERT 토크나이저 - wordpiece   
vocab.txt 파일

In [ ]:
#BERT 토크나이저 선언
from transformers import BertTokenizer 
tokenizer_bert = BertTokenizer.from_pretrained(
    "/gdrive/My Drive/nlpbook/wordpiece",
    do_lower_case = False,
)

file /gdrive/My Drive/nlpbook/wordpiece/config.json not found


In [ ]:
f = open("/content/train.txt",'r')
lines = f.readlines()
line_list = []
for line in lines[:4]:
    line_list.append(line)
line_list

['아 더빙.. 진짜 짜증나네요 목소리\n',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나\n',
 '너무재밓었다그래서보는것을추천한다\n',
 '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정\n']

In [ ]:
#토큰화하기
sentences = line_list
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

In [ ]:
tokenized_sentences

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터',
  '##보고',
  '초딩',
  '##영화',
  '##줄',
  '.',
  '.',
  '.',
  '.',
  '오버',
  '##연기',
  '##조차',
  '가볍',
  '##지',
  '않',
  '##구나'],
 ['[UNK]'],
 ['교',
  '##도',
  '##소',
  '이야기',
  '##구먼',
  '.',
  '.',
  '솔직히',
  '재미는',
  '없다',
  '.',
  '.',
  '평점',
  '조',
  '##정']]

In [ ]:
#BERT 모델 입력 만들기
batch_inputs = tokenizer_bert(
    sentences, 
    padding = "max_length",
    max_length = 12,
    truncation = True,
)

In [ ]:
batch_inputs

{'input_ids': [[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0], [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1199, 16, 3], [2, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 156, 1014, 1253, 2111, 9393, 16, 16, 2245, 2942, 2080, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


** 3가지 입력값이 만들어짐  
- input_ids : 토큰 인덱스 시퀀스를 나타냄
  - BERT는 [CLS],[SEP] 라는 토큰을 사용- 문장 시작과 끝에 이 토큰을 덧붙임 
  - input_ids [CLS]:2, [SEP]:3
 
- attention_mask : 일반토큰이 자리한 곳(1), 패딩 토큰이 자리한 곳(2)  
- token_type_ids : segment에 해당하는 것  
  - BERT모델은 기본적으로 문서(문장)을 2개를 입력 받는데, 이 두개는 token_type_ids로 구분함 !
  - 첫번째 세그먼트에 해당하는 문장의 token_type_ids는 0, 두번쨰 세그멘트는 1.
  - 이번 예제에서는 문장을 하나씩만 넣었으므로, 모두 0으로 처리됨
